# FHI Module 7 Hands-on --Introduction


## 1. Introduction of the data

[MIMIC-II](https://physionet.org/mimic2/mimic2_clinical_overview.shtml) is a freely available database of ICU patients. To access the full database (now migrated to [MIMIC-III](https://www.nature.com/articles/sdata201635.pdf))  you must sign a data use agreement. However, there is a [demo data set](https://physionet.org/mimic2/demo/) based on 4000 deceased patients that can be used without signing any DUA [1].

### 1.1 How to Use the MIMIC-II Database
* [MIMIC-II Cookbook](https://physionet.org/mimic2/demo/MIMICIICookBook_v1.pdf)
* [MIMIC Data Dictionaries](http://physionet.incor.usp.br/physiobank/database/dictionaries/)


### 1.2 The Varieties of...Data
The data set is very rich and so is a good resource for exploring the varieties of clinical data

[database schemas at glance](https://mit-lcp.github.io/mimic-schema-spy/)

Data incluces free text notes (nursing, radiology, discharg summaries, etc.), input/output events, test results, procedure codes, diagnosis codes, etc.

### 1.3 How did we prepare this dataset

1) **Retrieved all the MD notes**    
    Because the family history is usually recorded by doctors. We want to find more *positive mentions of the target concepts* when creating this dataset. Why?   
    In this dataset, there are only two types of physician notes: 'discharge summary' and 'MD notes.' The latter only has a few and doesn't have any family history mentions. So we only retrieve discharge summaries.  
    More details of how to, can be found [Retrieve the notes from MIMIC db](advanced/Retrieve the notes from MIMIC db.ipynb)

2) **Further narrowed down the notes**    
    Used text search to identify notes that have relevant phrases like *'breast cancer', 'colon cancer', etc. (needs to be rich and complete).   
    
3) **Randomly pulled notes from the rest as negative samples**    
    Manually went over the notes to make sure that there are no mentions of our interest.
    
4) **Preprocessed the documents**  
    Replaced the PHI markups with mimic real information. For example, replaced '[\*\* Known patient lastname \*\*]' in an original document with a random name 'Adams'.  
    
5) **Annotated the dataset**    
    Annotated the dataset using [Brat](http://brat.nlplab.org/).
    
6) **Packaged up**    
    Split the dataset through group sampling into training (60%) and testing (40%). Zipped them into two zip files. More details of how to do it, can be found [SplitDataset.ipynb](advanced/SplitDataset.ipynb)




## 2. Explore the data

Let's take a look at the annotated data, see how it looks like.

In [ ]:
# import packages that we will need
from nlp_pneumonia_utils import read_doc_annotations
from nlp_pneumonia_utils import DocumentClassifier
from nlp_pneumonia_utils import list_errors
from visual import snippets_markup
from visual import view_pycontext_output
from visual import display_doc_text
# packages for interaction
from IPython.display import display, HTML
import ipywidgets

### 2.1 Load our training set

In [ ]:
# first we need to tell which type is considered for positive document
pos_doc_type='FAM_BREAST_CA_DOC'

annotated_doc_map = read_doc_annotations(archive_file='data/bc_train.zip', pos_type=pos_doc_type)

# let's also use a simple list of documents as well as this map
annotated_docs = list(annotated_doc_map.values())

print('Total Annotated Documents : {0}'.format(len(annotated_docs)))

### 2.2 See what have been annotated

In [ ]:
#check notes have positive mentions

# limit the number of documents to display

total_display=15

pos_docs=dict((k, v) for k, v in annotated_doc_map.items() if  v.annotations[0].type ==pos_doc_type)

display(HTML(snippets_markup(dict(list(pos_docs.items())[:total_display]),'FAM_BREAST_CA', 900,400)))

See where the snippets came from:

In [ ]:
display_doc_text(pos_docs,300)

### 2.3 See negative documents

In [ ]:
neg_docs=dict((k, v) for k, v in annotated_doc_map.items() if  v.annotations[0].type !=pos_doc_type)

# comment the line below to clear the output cell
display_doc_text(neg_docs,300)

## 3. A simple NLP system to identify mentions of family history of breast cancer:  pyConText 

In short, pyConText will take in a sentence string ("mother had breast cancer"), then:

1. Locate the target terms ("breast cancer")
2. Look for context clues around the target terms, and assign modifier values to them ("mother" --> family history of breast cancer)

Check the [pyConText Notebook here](2.FHI_Project_Intro_pyConText.ipynb)


## 4. Going from pyConText output to a document classification

What rules might you create to classify a report with pyConText output as FAM_BREAST_CA_DOC or NEG_BREAST_CA_DOC?


###  Here are the steps:

1. Check these modifier values and rename the annotation type as necessary (See: [feature inference rule file](/edit/KB/fam_bc_featurer_inferences.csv))
2. Draw document level conclusion based on the evidence annotation types. If no evidence annotation exists, make a default conclusion (the last line of the [document inference rule file](/edit/KB/fam_bc_doc_inferences.csv))

For more detailed implementation, please check the <span style="color:blue">FeatureInferencer</span> class and the <span style="color:blue">DocumentInferencer</span> class in [this DocumentClassifier.py](/edit/DocumentClassifier.py)



## 5. Wrap everything up in DocumentClassifier

We have created a DocumentClassifier, which can take a document text:
1. Segment document text into sentences
2. Apply pyConText
3. Make document classification based on pyConText output  

For more detailed implementation, please check [this DocumentClassifier.py](/edit/DocumentClassifier.py) out. It does all the dirty work for you, and offers a few very clean and simple interfaces for you to use.

## 5. How do we know if the NLP performs well or poorly?

We use quantitative measurements to evaluate the performance of an NLP solution. Three measurements are commonly used:
precision, recall and F measure. Check this 16min video before the class:
<a href="https://www.youtube.com/embed/2akd6uwtowc?ecver=2"
target="_blank"><img src="http://img.youtube.com/vi/2akd6uwtowc/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="250" height="140" border="1" /></a>

## References:
1. MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available at: http://www.nature.com/articles/sdata201635

<br/><hr/>This material presented as part of the Foundations of Healthcare Informatics Course, 2017 Fall, BMI, University of Utah. It's revised from the <a href="https://github.com/UUDeCART/decart_rule_based_nlp">material</a> of the DeCART  Summer Program (Data, exploration, Computation, and Analytics Real-world Training for the Health Sciences) at the University of Utah in 2017. <br/><br/>Original presenters : Dr. Wendy Chapman, Jianlin Shi and Kelly Peterson.<br/>
Revised by: Jianlin Shi and Dr. Wendy Chapman<br/>
<img align="left" src="https://wiki.creativecommons.org/images/1/10/Cc.org_cc_by_license.jpg" alt="Except where otherwise noted, this website is licensed under a Creative Commons Attribution 3.0 Unported License.">